In [2]:
import xarray as xr
import glob
import datetime

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import numpy as np

from google.cloud import storage
import gcsfs
import zarr

import xskillscore as xs
import pandas as pd

import os

import re

In [145]:
def transform_dataset(meso, start_hour, end_hour):
    # Convert 10 minutes to hours (since rain intensity is in m/s)
    time_delta_hours = 10 / 60  # 10 minutes = 1/6 hours

    # Define the rolling window size, which corresponds to 6 hours
    rolling_window_size = int(6 / time_delta_hours)  # 36 time steps

    # Calculate the rolling sum over the last 6 hours and convert to mm
    total_precipitation_6hr_mm = (
        meso['prate']
        .rolling(time=rolling_window_size, center=False)
        .sum() * time_delta_hours * 3600 * 1000  # m/s to mm
    )

    # Add this as a new variable in the dataset
    meso['total_precipitation_6hr_meso'] = total_precipitation_6hr_mm

    # Define the date range based on your dataset's time dimension
    date_range = pd.date_range(start='2022-01-01', end='2023-01-01', freq='D')[:-1]

    # Extract the time values for each day (144 points per day, 10-minute intervals)
    time_points = pd.date_range('2022-01-01 00:00', periods=144, freq='10min')

    # Transform each variable with a time dimension
    data_vars = {
        var_name: (
            ('date', 'time') + var_data.dims[1:], 
            var_data.values.reshape((len(date_range), len(time_points)) + var_data.shape[1:])
        )
        if 'time' in var_data.dims else var_data
        for var_name, var_data in meso.data_vars.items()
    }

    # Create the new dataset with date and time dimensions
    meso = xr.Dataset(
        data_vars=data_vars,
        coords={
            'date': date_range,
            'time': time_points,
            'height_above_ground_level': meso.coords['height_above_ground_level'],
            'station': meso['id'].data,
        }
    )

    # Drop unnecessary variables and coordinates
    meso = meso.drop_vars(['id', 'xf', 'yf', 'i', 'j', 'height_above_base_level'])

    # Slice for hourly values (every hour)
    meso = meso.sel(time=meso['time'].dt.minute == 0)

    # Rename 'time' to 'prediction_timedelta' and convert to timedelta64
    meso = meso.rename({'time': 'prediction_timedelta'})
    prediction_timedelta = (
        meso['prediction_timedelta'].astype('datetime64[ns]')
        .astype('timedelta64[ns]') % np.timedelta64(1, 'D')
    )

    # Adjust prediction_timedelta for start and end hours
    prediction_timedelta += np.timedelta64(start_hour, 'h')

    # Assign the adjusted prediction_timedelta back to the dataset
    meso = meso.assign_coords(prediction_timedelta=prediction_timedelta)

    # Rename 'date' to 'time'
    meso = meso.rename({'date': 'time'})

    # Slice prediction_timedelta for every 6 hours within the specified range
    meso = meso.sel(prediction_timedelta=meso['prediction_timedelta'].isin(
        [np.timedelta64(i, 'h') for i in range(start_hour, end_hour + 1, 6)]
    ))

    # Slice for start date 2022-01-12
    meso = meso.sel(time=slice('2022-01-12', None))

    # Extract 2m temperature and 10m wind components
    meso['2m_temperature_meso'] = meso['TC'].sel(height_above_ground_level=2)
    meso['10m_u_component_of_wind_meso'] = meso['u'].sel(height_above_ground_level=10)
    meso['10m_v_component_of_wind_meso'] = meso['v'].sel(height_above_ground_level=10)

    # Convert surface level pressure to mean sea level pressure and convert to hPa
    meso['mean_sea_level_pressure_meso'] = (
    (meso['ps'] / 100) + 
    meso['height_above_sea_level'].sel(height_above_ground_level=2) - 
    2 * 9.81 / 1000
)

    # Drop original variables and unnecessary coordinates
    meso = meso.drop_vars(['TC', 'u', 'v', 'ps', 'M', 'phi', 'prate', 'height_above_sea_level', 'height_above_ground_level'])

    # Slice for all prediction_timedelta values larger than 0
    meso = meso.sel(prediction_timedelta=meso['prediction_timedelta'] > np.timedelta64(0, 'h'))

    return meso

def add_96_hour_lead_time(meso):
    # Identify the last time step (T23:50) in the dataset
    last_time_step = meso.isel(prediction_timedelta=-1)

    # Duplicate the last time step to create a new time step for T24:00
    new_time_step = last_time_step.copy(deep=True)
    new_time_step = new_time_step.assign_coords(prediction_timedelta=np.timedelta64(96, 'h'))

    # Concatenate the new time step to the existing dataset
    meso_extended = xr.concat([meso, new_time_step], dim='prediction_timedelta')

    return meso_extended

# Load the datasets
meso_0_24 = xr.open_dataset('/net/shared/student-projects/koenr/mesograsp_2022/extractorOutTFMetmast.000_leadtime-00-24.nc')
meso_24_48 = xr.open_dataset('/net/shared/student-projects/koenr/mesograsp_2022/extractorOutTFMetmast.000_leadtime-24-48.nc')
meso_48_72 = xr.open_dataset('/net/shared/student-projects/koenr/mesograsp_2022/extractorOutTFMetmast.000_leadtime-48-72.nc')
meso_72_96 = xr.open_dataset('/net/shared/student-projects/koenr/mesograsp_2022/extractorOutTFMetmast.000_leadtime-72-96.nc')

# Apply transformations
meso_0_24_transformed = transform_dataset(meso_0_24, start_hour=0, end_hour=24)
meso_24_48_transformed = transform_dataset(meso_24_48, start_hour=24, end_hour=48)
meso_48_72_transformed = transform_dataset(meso_48_72, start_hour=48, end_hour=72)
meso_72_96_transformed = transform_dataset(meso_72_96, start_hour=72, end_hour=96)

# Add the missing 96-hour lead time to the 72-96 dataset
meso_72_96_extended = add_96_hour_lead_time(meso_72_96_transformed)

# Concatenate the datasets along the prediction_timedelta dimension
meso_combined = xr.concat([
    meso_0_24_transformed, 
    meso_24_48_transformed, 
    meso_48_72_transformed, 
    meso_72_96_extended
], dim='prediction_timedelta')

# Ensure 'station' is correctly aligned with 'index'
if 'index' in meso_combined.coords and 'station' in meso_combined.coords:
    meso_combined = meso_combined.assign_coords(station=('index', meso_combined['station'].values))

# Now swap 'index' with 'station' and drop 'index'
meso_combined = meso_combined.swap_dims({'index': 'station'}).drop_vars('index')
# Final combined dataset
meso_combined


<xarray.Dataset> Size: 5MB
Dimensions:                       (time: 354, prediction_timedelta: 16,
                                   station: 48)
Coordinates:
  * time                          (time) datetime64[ns] 3kB 2022-01-12 ... 20...
  * prediction_timedelta          (prediction_timedelta) timedelta64[ns] 128B ...
  * station                       (station) <U5 960B '06201' '06203' ... '06212'
Data variables:
    total_precipitation_6hr_meso  (time, prediction_timedelta, station) float32 1MB ...
    2m_temperature_meso           (time, prediction_timedelta, station) float32 1MB ...
    10m_u_component_of_wind_meso  (time, prediction_timedelta, station) float32 1MB ...
    10m_v_component_of_wind_meso  (time, prediction_timedelta, station) float32 1MB ...
    mean_sea_level_pressure_meso  (time, prediction_timedelta, station) float32 1MB ...

In [133]:
# Choose variables and times for comparison
variables_to_check = ['2m_temperature', '10m_u_component_of_wind', 'total_precipitation_6hr']
times_to_check = ['2022-01-13', '2022-01-15']

# Define prediction times for 0-24 and 24-48 hours
prediction_times_0_24 = [ '6h', '12h', '18h']
prediction_times_24_48 = ['24h', '30h', '36h', '42h']

# Function to compare values
def compare_values(dataset1, dataset2, var, time, prediction_timedelta):
    value1 = dataset1[var].sel(time=time, prediction_timedelta=prediction_timedelta).values
    value2 = dataset2[var].sel(time=time, prediction_timedelta=prediction_timedelta).values
    return np.array_equal(value1, value2)

# Iterate over the selected variables, times, and prediction times
for var in variables_to_check:
    for time in times_to_check:
        # Compare values for the 0-24 hour predictions
        for pred_time in prediction_times_0_24:
            if compare_values(meso_combined, meso_0_24_transformed, var, time, pred_time):
                print(f"Values for {var} at time {time} and prediction_timedelta={pred_time} match in meso_0_24.")
            else:
                print(f"Mismatch found for {var} at time {time} and prediction_timedelta={pred_time} in meso_0_24.")
        
        # Compare values for the 24-48 hour predictions
        for pred_time in prediction_times_24_48:
            if compare_values(meso_combined, meso_24_48_transformed, var, time, pred_time):
                print(f"Values for {var} at time {time} and prediction_timedelta={pred_time} match in meso_24_48.")
            else:
                print(f"Mismatch found for {var} at time {time} and prediction_timedelta={pred_time} in meso_24_48.")


Values for 2m_temperature at time 2022-01-13 and prediction_timedelta=6h match in meso_0_24.
Values for 2m_temperature at time 2022-01-13 and prediction_timedelta=12h match in meso_0_24.
Values for 2m_temperature at time 2022-01-13 and prediction_timedelta=18h match in meso_0_24.
Values for 2m_temperature at time 2022-01-13 and prediction_timedelta=24h match in meso_24_48.
Values for 2m_temperature at time 2022-01-13 and prediction_timedelta=30h match in meso_24_48.
Values for 2m_temperature at time 2022-01-13 and prediction_timedelta=36h match in meso_24_48.
Values for 2m_temperature at time 2022-01-13 and prediction_timedelta=42h match in meso_24_48.
Values for 2m_temperature at time 2022-01-15 and prediction_timedelta=6h match in meso_0_24.
Values for 2m_temperature at time 2022-01-15 and prediction_timedelta=12h match in meso_0_24.
Values for 2m_temperature at time 2022-01-15 and prediction_timedelta=18h match in meso_0_24.
Values for 2m_temperature at time 2022-01-15 and predictio

In [147]:
meso_combined.to_netcdf('meso_2022.nc')

In [144]:
meso_24_48['ps']

<xarray.DataArray 'ps' (time: 52560, index: 48)> Size: 10MB
array([[100670.58 , 101122.42 , 100954.36 , ..., 101508.195, 100569.08 ,
        101062.44 ],
       [100661.13 , 101111.88 , 100944.086, ..., 101498.75 , 100561.39 ,
        101053.04 ],
       [100652.34 , 101101.39 , 100933.78 , ..., 101494.07 , 100558.47 ,
        101044.18 ],
       ...,
       [100392.805, 100707.52 , 100594.86 , ..., 100998.37 , 100040.51 ,
        100662.625],
       [100399.64 , 100710.766, 100599.85 , ..., 100994.914, 100037.11 ,
        100666.734],
       [100406.45 , 100715.055, 100604.05 , ..., 100992.2  , 100034.08 ,
        100672.6  ]], dtype=float32)
Coordinates:
  * index    (index) int32 192B 0 1 2 3 4 5 6 7 8 ... 39 40 41 42 43 44 45 46 47
  * time     (time) datetime64[ns] 420kB 2022-01-02T00:10:00 ... 2023-01-02
Attributes:
    long_name:  terrain following surface pressure 
    units:      Pa